In [43]:
#search tweets according to search target，do preference analysis
import twitter
import json
import time
from textblob import TextBlob

def oauth_login():
    # Go to http://twitter.com/apps/new to create an app and get values
    # for these credentials that you'll need to provide in place of these
    # empty string values that are defined as placeholders.
    # See https://dev.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = '5Pd25XKxJkdNUIm0LoDGxfNTB'
    CONSUMER_SECRET ='KQodfRCfkvDdDuQtafFwwvrsS1Q9uk5t3CznR9T5UkP3nwOdDu'
    OAUTH_TOKEN = '909871523447607298-G483xW9bSLv2RqZYlr3AZ4yb9gnUW7A'
    OAUTH_TOKEN_SECRET = '1DQ2DNiGip1IzhYJEbry1HAGKB2bPXE0HDOFIQs5Uouqw'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

search_target = [['Citi Bike','New York','NYC'],['Capital Bike','Washington','DC'],['Citi Bike','Miami',''],['Divvy','Chicago',''],
                 ['Hubway','Boston',''],['lime','Seattle',''],['lime','Dallas',''],['lime','San Francisco','"Bay Area"'],
                 ['lime','South Lake Tahoe',''],['lime','Key Biscayne',''],['lime','South Bend',''],['lime','Greensboro',''],
                 ['lime','Imperial Beach',''],['ofo','Seattle',''],['ofo','Worcester',''],['SpinBike','Seattle',''],
                 ['SpinBike','Washington','DC'],['Mobike','Washington','DC'],['bluegogo','San Francisco','"Bay Area"']]

sentiment = []

count = 100

for item in search_target:
    q = item[0]+' '+item[1]       
    #begin search
    search_results = twitter_api.search.tweets(q=q, count=count)
    statuses = search_results['statuses']

    for _ in range(5):
      #  print("Length of statuses", len(statuses))
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError: # No more results when next_results doesn't exist
            break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    
        search_results = oauth_login().search.tweets(**kwargs)
        statuses += search_results['statuses']
        
    status_texts = [ status['text'] 
                     for status in statuses ]
    text = json.dumps(status_texts, indent = 1)

    blob1 = TextBlob(text)
    i = [q,len(statuses),blob1.sentiment.polarity]

    sentiment.append(i)
    

In [44]:
print(sentiment)
# the first part of the result is the key word,.
# the second part is the point of preference analysis, the higher the point is, the more they prefer the key word.

[['Citi Bike New York', 200, 0.5161317953403365], ['Capital Bike Washington', 4, 0.21428571428571427], ['Citi Bike Miami', 5, 0.4600000000000001], ['Divvy Chicago', 30, 0.41015625], ['Hubway Boston', 31, 0.39263975966562176], ['lime Seattle', 27, 0.1025974025974026], ['lime Dallas', 11, 0.11750000000000001], ['lime San Francisco', 6, 0.16], ['lime South Lake Tahoe', 0, 0.0], ['lime Key Biscayne', 0, 0.0], ['lime South Bend', 0, 0.0], ['lime Greensboro', 2, 0.0], ['lime Imperial Beach', 2, 0.0], ['ofo Seattle', 115, 0.16600328947368417], ['ofo Worcester', 97, 0.32120329352850546], ['SpinBike Seattle', 0, 0.0], ['SpinBike Washington', 0, 0.0], ['Mobike Washington', 141, 0.23143552875695741], ['bluegogo San Francisco', 0, 0.0]]


In [12]:
words = [ w 
          for t in status_texts 
              for w in t.split() ]

from collections import Counter
import re

w = re.split(r'[\s\,\;\:\n\"\[\]\\\#\.]+', json.dumps(words,indent=1)) #Use regular expression to get a clear word list

In [13]:
#Get American Cities list
cities = []
f  = open('/Users/weiqingli/Downloads/PEP_2016_PEPANNRSIP.US12A/americacities.csv')
for line in f.readlines():
    line = line.strip('\n')
    line = line.rstrip()
    cities.append(line)
f.close()



In [15]:
#Analysis of the counts of each city that was refered in the tweets
from prettytable import PrettyTable

word_city = []

for i in w:
    if i in cities:
        word_city.append(i)

pt = PrettyTable(['City name','Count']) 
c = Counter(word_city)
[ pt.add_row(kv) for kv in c.most_common()]
pt.align['City name'], pt.align['Count'] = 'l', 'r' # Set column alignment
print(pt)

+------------+-------+
| City name  | Count |
+------------+-------+
| Seattle    |    13 |
| Washington |     9 |
| Arlington  |     5 |
| Baltimore  |     5 |
| Rochester  |     4 |
| Orange     |     3 |
| Savannah   |     3 |
| National   |     3 |
| Hoboken    |     2 |
| Georgetown |     1 |
| Meridian   |     1 |
| Manchester |     1 |
| Manhattan  |     1 |
| Boston     |     1 |
| League     |     1 |
| Raleigh    |     1 |
| Union      |     1 |
| Layton     |     1 |
+------------+-------+
